In [ ]:
import os
import json
import functools

import numpy
import matplotlib
from matplotlib import pyplot

from discohisto import (
    fit_normal,
    fit_cabinetry,
    fit_cabinetry_post,
    fit_linspace,
    fit_mcmc_mix,
    fit_mcmc_tfp_ham,
    region,
    limit,
)

In [ ]:
SEARCHES_PATH = "searches/"

In [ ]:
@functools.cache
def load_searches():
    searches = []
    for item in os.scandir(SEARCHES_PATH):
        if not item.is_dir():
            continue
        searches.append(item.name)
        
    return sorted(searches)


def load_reported(search):
    path = os.path.join(SEARCHES_PATH, search, "reported.json")
    with open(path) as file_:
        reported = json.load(file_)
    return reported


def _get_n_region(reg):
    sr_name = reg.signal_region_name
    observations = reg.workspace

    for obs in reg.workspace["observations"]:
        if obs["name"] == sr_name:
            return obs["data"][0]

    raise ValueError(sr_name)


def _load_mcmc_limits(path, *, suffix):
    mcmc_types = ["mix", "tfp_ham"]
    lim = None
    for mcmc_type in mcmc_types:
        suffix_i = "_mcmc_%s_%s" % (mcmc_type, suffix)
        try:
            lim = limit.LimitScan.load(path, suffix=suffix_i)
        except FileNotFoundError:
            ...
    assert lim is not None
    return lim

In [ ]:
def load_frame():
    searches = load_searches()
    
    # frame entries
    search_ = []
    region_ = []
    reported_n = []
    reported_bkg = []
    reported_bkg_hi = []
    reported_bkg_lo = []
    reported_s95obs = []
    reported_s95exp = []
    reported_s95exp_hi = []
    reported_s95exp_lo = []
    
    region_n = []
    
    fit_cabinetry_bkg = []
    fit_cabinetry_err = []
    fit_cabinetry_post_bkg = []
    fit_cabinetry_post_err = []
    
    limit_cabinetry_obs_2 = []
    limit_cabinetry_obs_3 = []
    
    limit_mcmc_obs_2 = []
    limit_mcmc_obs_3 = []
    
    for search in searches:
        reported = load_reported(search)
        for region_name in reported:
            search_.append(search)
            region_.append(region_name)
            
            # reported
            reported_reg = reported[region_name]
            reported_n.append(reported_reg["n"])
            reported_bkg.append(reported_reg["bkg"])
            reported_bkg_hi.append(reported_reg["bkg_hi"])
            reported_bkg_lo.append(reported_reg["bkg_lo"])
            reported_s95obs.append(reported_reg["s95obs"])
            reported_s95exp.append(reported_reg["s95exp"])
            reported_s95exp_hi.append(reported_reg["s95exp_hi"])
            reported_s95exp_lo.append(reported_reg["s95exp_lo"])
            
            # region
            region_dir = os.path.join(SEARCHES_PATH, search, region_name)
            region_i = region.Region.load(region_dir)
            region_n.append(_get_n_region(region_i))
            
            # standard fits
            fit_dir = os.path.join(region_dir, "fit")
            
            fit = fit_cabinetry.FitCabinetry.load(fit_dir)
            fit_cabinetry_bkg.append(fit.yield_pre)
            fit_cabinetry_err.append(fit.error_pre)
            
            fit = fit_cabinetry_post.FitCabinetryPost.load(fit_dir)
            fit_cabinetry_post_bkg.append(fit.yield_post)
            fit_cabinetry_post_err.append(fit.error_post)
            
            # limits
            limit_dir = os.path.join(fit_dir, "limit")
            
            lim = limit.LimitScan.load(limit_dir, suffix="_cabinetry_observed")
            assert lim.levels[6:8] == [-2, -3], lim.levels[6:8]
            limit_cabinetry_obs_2.append(lim.points[6][-1])
            limit_cabinetry_obs_3.append(lim.points[7][-1])
            
            lim = _load_mcmc_limits(limit_dir, suffix="observed")
            assert lim.levels[6:8] == [-2, -3], lim.levels[6:8]
            limit_mcmc_obs_2.append(lim.points[6][-1])
            limit_mcmc_obs_3.append(lim.points[7][-1])
            
    
    return dict(
        # labels
        search_=search_,
        region_=region_,
        # reported
        reported_n=reported_n,
        reported_bkg=reported_bkg,
        reported_bkg_hi=reported_bkg_hi,
        reported_bkg_lo=reported_bkg_lo,
        reported_s95obs=reported_s95obs,
        reported_s95exp=reported_s95exp,
        reported_s95exp_hi=reported_s95exp_hi,
        reported_s95exp_lo=reported_s95exp_lo,
        region_n=region_n,
        # fits
        fit_cabinetry_bkg=fit_cabinetry_bkg,
        fit_cabinetry_err=fit_cabinetry_err,
        fit_cabinetry_post_bkg=fit_cabinetry_post_bkg,
        fit_cabinetry_post_err=fit_cabinetry_post_err,
        # limits
        limit_cabinetry_obs_2=limit_cabinetry_obs_2,
        limit_cabinetry_obs_3=limit_cabinetry_obs_3,
        limit_mcmc_obs_2=limit_mcmc_obs_2,
        limit_mcmc_obs_3=limit_mcmc_obs_3,
    )
        

FRAME = load_frame()

In [ ]:
print(FRAME["reported_n"] == FRAME["region_n"])

In [ ]:
print(load_searches())

In [ ]:
def plot_bkgs():
    repored_bkg = numpy.array(FRAME["reported_bkg"])
    fit_cabinetry_bkg = numpy.array(FRAME["fit_cabinetry_bkg"])

    y = fit_cabinetry_bkg / repored_bkg
    x = numpy.arange(len(y)) + 0.5
    
    pyplot.scatter(x, y, lw=0, s=2, marker=",")
    
    pyplot.ylim(0, 2)
    pyplot.show()
    
plot_bkgs()

In [ ]:
def plot_bkgs_post():
    repored_bkg = numpy.array(FRAME["reported_bkg"])
    fit_cabinetry_bkg = numpy.array(FRAME["fit_cabinetry_post_bkg"])

    y = fit_cabinetry_bkg / repored_bkg
    x = numpy.arange(len(y)) + 0.5
    
    pyplot.scatter(x, y, lw=0, s=2, marker=",")
    
    pyplot.ylim(0, 2)
    pyplot.show()
    
plot_bkgs_post()

In [ ]:
def plot_limits():
    repored_obs = numpy.array(FRAME["reported_s95obs"])
    cabinetry_obs_2 = numpy.array(FRAME["limit_cabinetry_obs_2"])
    cabinetry_obs_3 = numpy.array(FRAME["limit_cabinetry_obs_3"])

    y2 = cabinetry_obs_2
    y3 = cabinetry_obs_3
    x = repored_obs
    
    pyplot.scatter(y2, x, color="r", lw=0, s=2, marker=",")
    pyplot.scatter(y3, x, color="b", lw=0, s=2, marker=",")
    pyplot.plot([0, 200], [0, 200], "k", alpha=0.2)
    
    pyplot.yscale("log")
    pyplot.xscale("log")
    pyplot.show()


plot_limits()

In [ ]:
def plot_limits_mcmc():
    repored_obs = numpy.array(FRAME["reported_s95obs"])
    mcmc_obs_2 = numpy.array(FRAME["limit_mcmc_obs_2"])
    mcmc_obs_3 = numpy.array(FRAME["limit_mcmc_obs_3"])

    y2 = mcmc_obs_2
    y3 = mcmc_obs_3
    x = repored_obs
    
    pyplot.scatter(y2, x, color="r", lw=0, s=2, marker=",")
    pyplot.scatter(y3, x, color="b", lw=0, s=2, marker=",")
    pyplot.plot([0, 200], [0, 200], "k", alpha=0.2)
    
    pyplot.yscale("log")
    pyplot.xscale("log")
    pyplot.show()


plot_limits_mcmc()